In [1]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv("terro.csv",encoding=("ISO-8859-1"))

C:\Users\ANJU\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [6]:
### How many attacks happened in India
data['country_txt'].unique().tolist()
data[data['country_txt']=="India"]
data[data['country_txt']=="India"].shape[0]

11960

In [7]:
## How many attacks happened in India and upto 3 people were killed
data[(data['country_txt']=='India')&(data['nkill']<=3)].shape[0]

10288

In [8]:
# Extract the city and summary for attacks above
data.query("country_txt=='India' & nkill<=3")[['city','summary']].head(10)

,city,summary
1186,New Delhi,NaN
3857,New Delhi,NaN
5327,Bombay,NaN
7342,Imphal,NaN
8685,Unknown,NaN
8831,Unknown,NaN
9053,Amapur,NaN
9113,Raisikah,NaN
9116,Champhai,NaN
9146,Aizawl,NaN


In [9]:
# In a single terror incident in India, find out top 5 cities by number killed
data.query("country_txt=='India'").sort_values('nkill',ascending=False)['city'].head(5)

81098                Mumbai
96726               Jhargam
54416                Bombay
95988    Dantewada district
56912              Banabari
Name: city, dtype: object

In [10]:
#### Adding new columns to the data
#Attacks which were successful and were suicide attacks
data['success_suicide']=data['suicide']+data['success']
data=data.assign(success_suicide_1=data['suicide']+data['success'])

data.query("success_suicide==2").shape[0]


5568

In [11]:

### Dropping columns
data.drop('success_suicide',axis=1) 
data.drop('success_suicide',axis=1,inplace=True) 

In [12]:
### Aggregations and manipulations using apply and map
#map: map a function to each element of a series object
data['nkill'].describe()
#Suppose we want to label all the incidents where the number killed was more than 5 as severe. This would involve applying a function on each element of the series, map helps in doing that
def get_label(x):
    if x>5:
        return 'Severe'
    else:
        return 'Not Severe'
data['nkill'].map(get_label)
#You can use lambda functions as well (if else follows a special form when used with lambdas )
data['nkill'].map(lambda x: "Severe" if x>5 else "Not Severe")

0         Not Severe
1         Not Severe
2         Not Severe
3         Not Severe
4         Not Severe
5         Not Severe
6         Not Severe
7         Not Severe
8         Not Severe
9         Not Severe
10        Not Severe
11        Not Severe
12        Not Severe
13        Not Severe
14        Not Severe
15        Not Severe
16        Not Severe
17        Not Severe
18        Not Severe
19        Not Severe
20        Not Severe
21        Not Severe
22        Not Severe
23        Not Severe
24        Not Severe
25        Not Severe
26        Not Severe
27        Not Severe
28        Not Severe
29        Not Severe
             ...    
181661    Not Severe
181662    Not Severe
181663    Not Severe
181664    Not Severe
181665    Not Severe
181666    Not Severe
181667    Not Severe
181668    Not Severe
181669    Not Severe
181670    Not Severe
181671    Not Severe
181672        Severe
181673        Severe
181674    Not Severe
181675    Not Severe
181676    Not Severe
181677    Not

In [13]:
# We can use apply to use a function row wise or column wise
# Let's write a function to label an incident that was both successful and suicidal

def get_label(row):
    if row['success']==1 and row['suicide']==1:
        return 1
    else:
        return 0
data.apply(get_label,axis=1)
data.apply(get_label,axis=1).unique()

data.apply(lambda row: 1 if row['success']==1 and row['suicide']==1 else 0,axis=1)


0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
181661    0
181662    0
181663    0
181664    0
181665    0
181666    0
181667    1
181668    0
181669    0
181670    0
181671    0
181672    1
181673    0
181674    0
181675    0
181676    0
181677    0
181678    0
181679    0
181680    0
181681    0
181682    0
181683    0
181684    0
181685    0
181686    0
181687    0
181688    0
181689    0
181690    0
Length: 181691, dtype: int64

In [14]:
### Create a new  category representing if the incident occured in Afghanistan, Pakistan or India as one level of the category and all the other countries as another level

def get_label(row):
    if row['country_txt']=='India' or row['country_txt']=='Afghnistan' or row['country_txt']=='Pakistan':
        return 'Af-Pak-India'
    else:
        return 'ROW'
data.apply(get_label,axis=1)

data['Local']=data.apply(get_label,axis=1)

In [15]:
data.apply(lambda row: 'Af-Pak-India' if row['country_txt']=='India' or row['country_txt']=='Afghnistan' or row['country_txt']=='Pakistan' else  'ROW', axis =1 )

0                  ROW
1                  ROW
2                  ROW
3                  ROW
4                  ROW
5                  ROW
6                  ROW
7                  ROW
8                  ROW
9                  ROW
10                 ROW
11                 ROW
12                 ROW
13                 ROW
14                 ROW
15                 ROW
16                 ROW
17                 ROW
18                 ROW
19                 ROW
20                 ROW
21                 ROW
22                 ROW
23                 ROW
24                 ROW
25                 ROW
26                 ROW
27                 ROW
28                 ROW
29                 ROW
              ...     
181661    Af-Pak-India
181662             ROW
181663    Af-Pak-India
181664             ROW
181665    Af-Pak-India
181666             ROW
181667             ROW
181668             ROW
181669             ROW
181670             ROW
181671             ROW
181672    Af-Pak-India
181673     

In [16]:

## Number of incidents in Af-Pak-India vs ROW
data['Local'].value_counts()
data.groupby('Local').size()

Local
Af-Pak-India     26328
ROW             155363
dtype: int64

In [17]:
## Number of suicide attacks by Af-Pak-India vs ROW
data.groupby(['Local','suicide']).agg({'eventid':np.size})
data.groupby(['Local','suicide'],as_index=False).agg({'eventid':np.size})

,Local,suicide,eventid
0,Af-Pak-India,0,25757
1,Af-Pak-India,1,571
2,ROW,0,149301
3,ROW,1,6062


In [18]:

## Number of suicides attacks and average kills by Af-Pak-India vs ROW
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size])
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size]).reset_index()

,Local,suicide,mean,size
0,Af-Pak-India,0,1.410930,25757.0
1,Af-Pak-India,1,13.126316,571.0
2,ROW,0,2.213852,149301.0
3,ROW,1,10.030395,6062.0


In [19]:
#Rename columns
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size]).reset_index().rename(columns={'mean':'Average_Kills','size':'Number_Incidents'})

data.groupby(["Local",'suicide']).agg({'nkill':np.mean,'nwound':np.sum}).reset_index().rename(columns={'nwound':'Total_Wounded','nkill':'Number_Killed'})


,Local,suicide,Number_Killed,Total_Wounded
0,Af-Pak-India,0,1.410930,56622.0
1,Af-Pak-India,1,13.126316,14396.0
2,ROW,0,2.213852,335768.0
3,ROW,1,10.030395,117083.0
